In [ ]:
%load_ext jupyter_black

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.dummy import DummyClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns

# Внутренние меры кластеризации

Или как оценивать, если правильных меток не существует в природе? 



In [ ]:
features, true_labels = make_blobs(
    n_samples=200, centers=3, cluster_std=2.75, random_state=42
)

kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(features)

## Кстати, кто такой Rand Index ?

print("KMeans Rand Index: ", metrics.rand_score(kmeans.labels_, true_labels))

In [ ]:
## А тут вы сколько ожидаете увидеть? А если пар -- 100?
random_labels = np.random.randint(0, 3, size=true_labels.shape)
print("Random Rand Index: ", metrics.rand_score(random_labels, true_labels))

In [ ]:
fig, (left, right) = plt.subplots(1, 2)

left.scatter(
    features[:, 0],
    features[:, 1],
    s=15,
    c=random_labels,
)

right.scatter(
    features[:, 0],
    features[:, 1],
    s=15,
    c=kmeans.labels_,
)

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

print(f"Silhouette score")
print(f"random clustering: {silhouette_score(features, random_labels)}")
print(f"kmeans: {silhouette_score(features, kmeans.labels_)}")

print()

print(f"David-Bouldin Index")
print(f"random clustering: {davies_bouldin_score(features, random_labels)}")
print(f"kmeans: {davies_bouldin_score(features, kmeans.labels_)}")

---

# Как выбрать число кластеров


### Elbow plot
Самый простой метод, продолжающий предыдущую тему.

Какое максимальное и минимальное значения $EV$? Как они достигаются?

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cd/DataClustering_ElbowCriterion.JPG" />

In [ ]:
# !pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(kmeans, k=(1, 9))

visualizer.fit(features)
visualizer.show()  # тут вместо explained variance сырые расстояния

### Silhouette analysis

Выше мы рассмотрели глобальный (усредненный) Silhouette score. Давайте теперь росмотрим на эти коэффициенты, сгруппированные по кластерам

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

# поменять число кластеров и посмотркть картинки
model = KMeans(2, random_state=42, n_init=10)
visualizer = SilhouetteVisualizer(model, colors="yellowbrick")

visualizer.fit(features)
visualizer.show()

---

## Case study: сжатие изображений

Адаптация примера из курса К.В. Воронцова.

Преобразуем изображение, приведя все значения
в интервал [0, 1]: можно использовать функцию `img_as_float` из модуля `skimage`.

In [ ]:
# ! pip install -q --upgrade scikit-image

In [ ]:
import numpy as np

import skimage
from skimage import data, io
from skimage.io import imread, imsave
from sklearn.cluster import KMeans

image = imread("Lenna.png")
print(image.shape, image.max(), image.min())

Каптинка в т.н. "полноцветном" формате, т.е. по 1 байту на каждый из трех каналов.

 **Сколько всего цветов ?**

In [ ]:
io.imshow(image)

**PSNR - peak signal-to-noise ratio**

Соотношение между максимумом возможного значения сигнала и мощностью шума, искажающего значения сигнала. Поскольку многие сигналы имеют широкий динамический диапазон, PSNR обычно измеряется в логарифмической шкале в децибелах.

PSNR наиболее часто используется для измерения уровня искажений при сжатии изображений. 

$$PSNR = 10 ~ \log_{10} \frac{MAX_{orig}^2}{MSE}$$

Довольно плохая метрика, кстати говоря; как думаете, в чем ее проблема?

In [ ]:
def psnr(X_true, X_clustered):

    mse = np.mean((X_true - X_clustered) ** 2)
    max_2 = np.max(X_true) ** 2

    return 10 * np.log10(max_2 / mse)

In [ ]:
psnr(image, image + np.ones_like(image) * 0.0000000000001)

Кластеризуем, и все пиксели, отнесенные в один кластер, попробуем заполнить двумя способами: медианным и средним цветом по кластеру. Таким образом мы "сжимаем" изображение. 

In [ ]:
float_image = skimage.img_as_float(image, force_copy=True)
float_image.shape,

Переведём в набор троек RGB

In [ ]:
RGB = ...

RGB.shape, RGB

In [ ]:
import os
os.makedirs('Lenna', exist_ok=True)

clusters = [25] + list(reversed(range(2, 10)))

for n_clusters in clusters:
    
    print(n_clusters, end=" ")
    
    # группируем цвета, которые есть в изображении
    model = KMeans(n_clusters=n_clusters, verbose=False, random_state=100, n_init=10)
    model.fit(RGB)
    X = RGB.copy()
    
    # запоминаем метки кластеров
    labels = model.labels_.T

    for cluster in range(n_clusters):
        
        # по каким индексам в списке пикселей живёт этот кластер?
        ...
        
        # заполняем заполненное средним
        ...
        
    # обратно в трёхмерный вид
    im = ...
    
    print(f"Frobenius norm: {np.linalg.norm(im - float_image):3.2f}" , end=" | ")    
    print(f"PSNR: {psnr(float_image, im):3.3f}" % psnr(float_image, im))
    
    rescaled_im = (im * 255).astype(np.uint8)
    
    # сохраняем
    imsave("Lenna/" + str(n_clusters) + ".png", rescaled_im)

In [ ]:
# image = imread("Lenna/6.png")
# io.imshow(image)

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i, c in enumerate(clusters):
    img = imread(f"Lenna/{c}.png")
    ax = fig.add_subplot(3, 3, i+1)
    ax.title.set_text(f'{c} clusters')
    plt.imshow(img)
plt.show()


---
## Case study 2: слова

In [ ]:
# ! wget http://vectors.nlpl.eu/repository/20/182.zip
# ! bash -c "unzip -o 182.zip"

Есть три файла - `meta.json` с метаданными, `lemma.num` с частотами слов и `model.txt` собственно с эмбеддингами.

Давайте кластеризовывать слова на основе их векторных представлений.

In [ ]:
! bash -c "tail -2 model.txt"

In [ ]:
# ! pip install nltk
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
STOPS = set(stopwords.words("russian"))
STOPS

In [ ]:
# выбрасываем стоп-слова
freq_list = ...
len(freq_list)

for i, item in enumerate(freq_list):
    print(item)
    if i == 5:
        break
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
tokens = []
raw_vectors = []

with open("model.txt", "r+", encoding="utf-8") as rf:

    # пропускаем первую строку
    next(rf)

    for line in tqdm(rf):
        # parse line, extract only vectors corresponding to frequent nouns
        ...


len(tokens), len(raw_vectors)

token2id = {t: i for i, t in enumerate(tokens)}
vectors = np.array(raw_vectors)

In [ ]:
# нормализуем?
vectors = ...

In [ ]:
vectors.shape, vectors.sum(axis=1).shape, vectors.sum(axis=1)

In [ ]:
from scipy.spatial.distance import cosine

from scipy.spatial.distance import cosine

king = raw_vectors[token2id["король"]]
queen = raw_vectors[token2id["королева"]]

man = raw_vectors[token2id["мужчина"]]
woman = raw_vectors[token2id["женщина"]]
cosine(king - man, queen - woman)

In [ ]:
from sklearn import cluster

mbk_means = cluster.MiniBatchKMeans(
    n_clusters=70,
    batch_size=1000,
    max_iter=10000,
    n_init=20,
    random_state=100,
    reassignment_ratio=0.1,
)
mbk_means.fit(vectors)

mbk_means.labels_

In [ ]:
tokens2clusters = {t: c for t, c in zip(tokens, mbk_means.labels_)}
cluster2tokens = {l: [] for l in mbk_means.labels_}

In [ ]:
for t, c in tokens2clusters.items():
    cluster2tokens[c].append(t)

for c in sorted(list(cluster2tokens)[:50]):
    print(f"\n>-- Cluster #{c}, {len(cluster2tokens[c])} objects.")
    print(" ".join([w for w in cluster2tokens[c][:100]]))

---
### Case study 3: dedublication

Довольно часто бывает нужно избавиться от неточных дубликатов коротких текстов, и никаких особых методов родом из предметной области (или внешних вспомогательных данных) для этого нет. Кроме того, как первое приближение для понимания, о чём вообще все эти короткие тексты, как их можно потом размечать, объединять в группы и так далее, -- может помочь кластеризация. 

In [ ]:
items = [w.strip() for w in open("fallout_possible_items.txt", "r+", encoding="utf-8").readlines() if w.strip()]
len(items)

In [ ]:
len(set(items))

In [ ]:
items = list(set(items))
sorted(items)[:10]

А нам точно понадобится машинное обучение?

In [ ]:
similarity = ...

In [ ]:
similarity[idx, :][:, idx]

Готовим своеобразный формат для `scipy.cluster`: треугольный кусок матрицы под диагональю, представленный списком

In [ ]:
distances = 1 - similarity
distances_prepared = []

for i in tqdm(range(distances.shape[0]), "distances matrix rows"):
    for j in range(distances.shape[0]):
        if i < j:
            distances_prepared.append(distances[i, j])

In [ ]:
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import linkage

inside_cluster_dist = 0.4

Z = linkage(np.array(distances_prepared), method="ward")
result = fcluster(Z, t=inside_cluster_dist, criterion="distance")

## Дендрограммы

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

# TODO: слишком много всего, надо нарисовать только часть входов

plt.figure(figsize=(8, 15))
dendrogram(Z, orientation="left", labels=items)
plt.show()

In [ ]:
clusters = {i: [] for i in result}

for i, idx in enumerate(result):
    clusters[idx].append(items[i])

In [ ]:
for cluster in clusters:
    print("\nCluster", cluster)
    print(clusters[cluster])

## Case study 4: Червяки

In [ ]:
import numpy as np

x, y = [], []

with open("worms/worms_2d.txt", "r+", encoding="utf-8") as rf:
    for line in rf:
        line = line.strip()
        if line:
            spl = line.split(" ")
            x.append(float(spl[0]))
            y.append(float(spl[1]))

x, y = np.array(x), np.array(y)
x.shape

In [ ]:
import matplotlib.pyplot as plt

plt.xlim(1500, 5500)
plt.ylim(2000, 5500)
plt.scatter(x, y, s=0.5, alpha=0.1)

In [ ]:
# еще у нас в этот раз есть правильные метки
labels = []

with open("worms/worms_2d-gt.pa", "r+", encoding="utf-8") as rf:
    for line in rf.readlines()[4:]:
        line = line.strip()
        if line:
            labels.append(int(line))

print("Labels count:", len(set(labels)))

labels = np.array(labels)

assert x.shape == labels.shape == y.shape

plt.xlim(1500, 5500)
plt.ylim(2000, 5500)
plt.scatter(x, y, s=0.5, alpha=0.05, c=labels, cmap="viridis")

Казалось бы, чего тут сложного... Давайте попробуем это кластеризовать, что ли

### Задание со звездочкой 17. Dunn index
Реализуйте Dunn Index. Будет засчитываться не абы-какая реализация, а соответствующая по стилю и оформлению реализации метрик в sklearn: обязательны докстринги, валидаторы и все такое.

То, как реализованы другие метрики, можно посмотреть [тут](https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/metrics/cluster/_unsupervised.py#L195).

In [ ]:
from sklearn.utils._param_validation import (
    validate_params,
)

@validate_params(...)
def dunn_score(...):
    """
    Compute the Dunn Index by given within-cluster distances (callable or precomputed) and
    between-cluster distances(callable or precomputed).
    ...
    
    """
    ...
    return

### Задание со звездочкой 18. Кластеризация текстов

Мы сделали кластеризацию **слов** на основе их векторных представлений. На самом деле, кластеризовывать тексты гораздо сложнее и интереснее; для этой задачи существуют специальные модели, эти модели обучаются на **корпусе текстов**. Мы поговорим о таких моделях в следующем семестре, и быстро обобщим их до **тематического моделирования**, но это будет потом. А что делать, если датасета с текстами нет? Наивный подход состоит в том, чтобы как-нибудь аггрегировать векторные представления слов, например взять их среднее.

Ваша задача - предложите способ кластеризации **предложений**, который бы использовал только векторные представления слов, реализуйте и продемонстрируйте, что ваш способ работает лучше, чем наивный. 

**Подсказка:** _того, что было рассказано сегодня, должно быть достаточно, нужно только грамотно скомпоновать разные части практики._